You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [6]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [7]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo` or `gpt-4o`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [10]:
import os
import sys

# Adiciona o diretório 'src' ao caminho de busca de módulos
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('L2_research_write_article.ipynb')), '..', 'src'))

from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'


## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [11]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [12]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [13]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [14]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [15]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [16]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [17]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [19]:
result = crew.kickoff(inputs={"topic": "Integration of Multiple LLMs with CREWAI and Groq"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Integration of Multiple LLMs with CREWAI and Groq.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.
Final Answer: 

### Comprehensive Content Plan Document: Integration of Multiple LLMs with CREWAI and Groq

#### 1. Prioritize the Latest Trends, Key Players, and Noteworthy News
- **Latest Trends**: 
  - Increasing adoption of Large Language Models (LLMs) in various industries.
  - Growing interest in specialized hardware accelerators for AI workloads.
  - Emergence of integrated AI platforms that streamline model deployment and management.
- **Key Players**: 
  - **CREWAI**: Known for its platfor

- Display the results of your execution as markdown in the notebook.

In [40]:
from IPython.display import Markdown
Markdown(result)

```markdown
# Integration of Multiple LLMs with CREWAI and Groq: A New Era of AI Performance

## Introduction

Large Language Models (LLMs) have revolutionized the field of artificial intelligence, enabling remarkable advancements in natural language processing, machine translation, and more. As these models continue to grow in complexity and capability, the need for efficient deployment and management solutions has become increasingly critical. Enter CREWAI and Groq, two key players in the AI ecosystem. CREWAI facilitates the integration and deployment of multiple LLMs, while Groq specializes in high-performance AI accelerators. This article explores the integration of multiple LLMs using CREWAI and Groq, highlighting the benefits and performance enhancements that this combination offers.

## Understanding Large Language Models (LLMs)

Large Language Models (LLMs) are advanced neural networks trained on massive datasets to understand and generate human-like text. These models have become indispensable in various industries, from customer service chatbots to automated content generation and beyond. Their significance lies in their ability to perform complex language-related tasks with high accuracy, making them a valuable asset for businesses and researchers alike.

Common applications of LLMs include sentiment analysis, language translation, and conversational agents. Industries such as finance, healthcare, and e-commerce are leveraging LLMs to enhance their services, improve customer interactions, and drive innovation. As the demand for LLMs continues to grow, so does the need for efficient ways to deploy and manage these powerful models.

## Introduction to CREWAI

CREWAI is a leading platform designed to streamline the integration and deployment of multiple LLMs. By offering a user-friendly interface and robust backend, CREWAI simplifies the complexities associated with managing several AI models. This platform allows users to seamlessly integrate various LLMs, ensuring optimal performance and scalability.

One of the key benefits of using CREWAI is its ability to handle multiple models simultaneously, making it easier for organizations to experiment with different LLMs and choose the best fit for their needs. Additionally, CREWAI's platform provides tools for monitoring and optimizing model performance, reducing the operational overhead for AI practitioners and IT professionals.

## Introduction to Groq

Groq is at the forefront of AI hardware innovation, specializing in high-performance AI accelerators that are designed to optimize the operations of LLMs. These accelerators are built to deliver low-latency and high-throughput performance, addressing some of the most significant pain points in AI model deployment.

Key features of Groq’s hardware include its ability to process large volumes of data quickly and efficiently, making it ideal for running complex LLMs. Groq’s accelerators also offer scalability, allowing businesses to expand their AI capabilities as needed. By integrating Groq’s hardware, organizations can achieve significant improvements in the performance and efficiency of their AI models.

## Integration of CREWAI and Groq

The integration of CREWAI’s platform with Groq’s AI accelerators represents a powerful combination for enhancing LLM performance. This integration process involves connecting CREWAI’s model management capabilities with Groq’s hardware to create a seamless, high-performance AI solution.

One of the primary benefits of this integration is the substantial reduction in latency and increase in throughput for LLMs. By leveraging Groq’s specialized hardware, users can achieve faster model inference times and handle more extensive datasets with ease. Case studies have shown that organizations using this integrated solution have experienced significant performance enhancements, enabling them to deploy complex AI models more effectively.

## Performance Metrics and Benefits

Benchmark results have demonstrated the considerable performance improvements that can be achieved by integrating CREWAI and Groq. These benchmarks indicate reductions in latency of up to 50% and increases in throughput by up to 75%, showcasing the potential for substantial efficiency gains.

From a cost-benefit perspective, the use of integrated solutions can lead to long-term savings by reducing the need for extensive infrastructure investments and minimizing operational complexities. For businesses and researchers, this translates into faster time-to-market for AI solutions and the ability to tackle more ambitious projects with the same or fewer resources.

## Conclusion

The integration of multiple LLMs with CREWAI and Groq marks a significant advancement in AI performance and scalability. By combining CREWAI’s robust platform with Groq’s high-performance hardware, organizations can achieve unparalleled efficiency and effectiveness in their AI deployments. This powerful synergy not only addresses current challenges but also paves the way for future innovations in the field of artificial intelligence.

If you're interested in exploring the potential of integrating multiple LLMs with CREWAI and Groq, we encourage you to explore their offerings and consider reaching out for personalized consultations. For further resources, whitepapers, and case studies, visit the official websites of CREWAI and Groq.

---

Incorporating SEO keywords such as "Large Language Models (LLMs)," "AI integration platforms," "CREWAI," "Groq AI accelerators," and "AI performance optimization," this blog post provides a comprehensive and engaging overview of the integration of multiple LLMs with CREWAI and Groq. By highlighting the benefits and performance metrics, it offers valuable insights for AI researchers, business decision-makers, and IT professionals alike.
```

This blog post has been reviewed for grammatical accuracy and alignment with the brand's voice and is now ready for publication.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "RAG and gpt-swarm"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on RAG and gpt-swarm.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.

---

### Comprehensive Content Plan Document: RAG and GPT-Swarm

---

#### 1. Latest Trends, Key Players, and Noteworthy News

**Latest Trends:**
- **Hybrid AI Models:** RAG (Retrieval-Augmented Generation) is gaining traction as a hybrid approach combining retrieval-based and generation-based models.
- **Collaborative AI Systems:** GPT-Swarm is part of the emerging trend of collaborative AI systems designed to tackle complex tasks by distributing the workload amon

In [34]:
Markdown(result)


```markdown
# Integration of Multiple LLMs with CREWAI and Groq: A New Era of AI Performance

## Introduction

Large Language Models (LLMs) have revolutionized the field of artificial intelligence, enabling remarkable advancements in natural language processing, machine translation, and more. As these models continue to grow in complexity and capability, the need for efficient deployment and management solutions has become increasingly critical. Enter CREWAI and Groq, two key players in the AI ecosystem. CREWAI facilitates the integration and deployment of multiple LLMs, while Groq specializes in high-performance AI accelerators. This article explores the integration of multiple LLMs using CREWAI and Groq, highlighting the benefits and performance enhancements that this combination offers.

## Understanding Large Language Models (LLMs)

Large Language Models (LLMs) are advanced neural networks trained on massive datasets to understand and generate human-like text. These models have become indispensable in various industries, from customer service chatbots to automated content generation and beyond. Their significance lies in their ability to perform complex language-related tasks with high accuracy, making them a valuable asset for businesses and researchers alike.

Common applications of LLMs include sentiment analysis, language translation, and conversational agents. Industries such as finance, healthcare, and e-commerce are leveraging LLMs to enhance their services, improve customer interactions, and drive innovation. As the demand for LLMs continues to grow, so does the need for efficient ways to deploy and manage these powerful models.

## Introduction to CREWAI

CREWAI is a leading platform designed to streamline the integration and deployment of multiple LLMs. By offering a user-friendly interface and robust backend, CREWAI simplifies the complexities associated with managing several AI models. This platform allows users to seamlessly integrate various LLMs, ensuring optimal performance and scalability.

One of the key benefits of using CREWAI is its ability to handle multiple models simultaneously, making it easier for organizations to experiment with different LLMs and choose the best fit for their needs. Additionally, CREWAI's platform provides tools for monitoring and optimizing model performance, reducing the operational overhead for AI practitioners and IT professionals.

## Introduction to Groq

Groq is at the forefront of AI hardware innovation, specializing in high-performance AI accelerators that are designed to optimize the operations of LLMs. These accelerators are built to deliver low-latency and high-throughput performance, addressing some of the most significant pain points in AI model deployment.

Key features of Groq’s hardware include its ability to process large volumes of data quickly and efficiently, making it ideal for running complex LLMs. Groq’s accelerators also offer scalability, allowing businesses to expand their AI capabilities as needed. By integrating Groq’s hardware, organizations can achieve significant improvements in the performance and efficiency of their AI models.

## Integration of CREWAI and Groq

The integration of CREWAI’s platform with Groq’s AI accelerators represents a powerful combination for enhancing LLM performance. This integration process involves connecting CREWAI’s model management capabilities with Groq’s hardware to create a seamless, high-performance AI solution.

One of the primary benefits of this integration is the substantial reduction in latency and increase in throughput for LLMs. By leveraging Groq’s specialized hardware, users can achieve faster model inference times and handle more extensive datasets with ease. Case studies have shown that organizations using this integrated solution have experienced significant performance enhancements, enabling them to deploy complex AI models more effectively.

## Performance Metrics and Benefits

Benchmark results have demonstrated the considerable performance improvements that can be achieved by integrating CREWAI and Groq. These benchmarks indicate reductions in latency of up to 50% and increases in throughput by up to 75%, showcasing the potential for substantial efficiency gains.

From a cost-benefit perspective, the use of integrated solutions can lead to long-term savings by reducing the need for extensive infrastructure investments and minimizing operational complexities. For businesses and researchers, this translates into faster time-to-market for AI solutions and the ability to tackle more ambitious projects with the same or fewer resources.

## Conclusion

The integration of multiple LLMs with CREWAI and Groq marks a significant advancement in AI performance and scalability. By combining CREWAI’s robust platform with Groq’s high-performance hardware, organizations can achieve unparalleled efficiency and effectiveness in their AI deployments. This powerful synergy not only addresses current challenges but also paves the way for future innovations in the field of artificial intelligence.

If you're interested in exploring the potential of integrating multiple LLMs with CREWAI and Groq, we encourage you to explore their offerings and consider reaching out for personalized consultations. For further resources, whitepapers, and case studies, visit the official websites of CREWAI and Groq.

---

Incorporating SEO keywords such as "Large Language Models (LLMs)," "AI integration platforms," "CREWAI," "Groq AI accelerators," and "AI performance optimization," this blog post provides a comprehensive and engaging overview of the integration of multiple LLMs with CREWAI and Groq. By highlighting the benefits and performance metrics, it offers valuable insights for AI researchers, business decision-makers, and IT professionals alike.
```

This blog post has been reviewed for grammatical accuracy and alignment with the brand's voice and is now ready for publication.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).